In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt

from multirtd.planners.simple_planner import Simple_Planner
import multirtd.params as params

In [ ]:
# Initialize planner
p_0 = np.array([[-5], [0]]) # This sets where the robot starts
LPM_file = os.path.join(os.getcwd(),'..', 'models', 'quadrotor_linear_planning_model.mat')
planner = Simple_Planner(LPM_file, p_0)

In [ ]:
# Lets add another instance
p_02 = np.array([[-5], [-2]]) # This sets where the robot starts
LPM_file = os.path.join(os.getcwd(),'..', 'models', 'quadrotor_linear_planning_model.mat')
planner2 = Simple_Planner(LPM_file, p_02)

In [ ]:
# Set planner goal and obstacles
planner.p_goal = np.array([[5], [0]]) # This sets the goal position
# (center, radius) tuples
static = [(np.array([3, 1]), 1),
          (np.array([0, -1.5]), 1),
          (np.array([-3, 1]), 1),
          (np.array([-3, -3]), 1)]
planner.obstacles = static

In [ ]:
# Other instance
planner2.p_goal = np.array([[5], [-.5]]) # This sets the goal position
# (center, radius) tuples
planner2.obstacles = static

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

#ax.scatter(p_0[0], p_0[1], s=50, marker='^', color='b')
ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

# Other instance
ax.add_patch(plt.Circle(tuple(p_02), params.R_BOT, color='b'))
ax.scatter(planner2.p_goal[0], planner2.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

plt.show()

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p = p_0
v = np.zeros((2,1))
a = np.zeros((2,1))
# State history
P = []
planning_times = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan((p,v,a))
        planning_times.append(time.time() - start_t)
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    p = trajectory[0][:,traj_idx][:,None]
    v = trajectory[1][:,traj_idx][:,None]
    a = trajectory[2][:,traj_idx][:,None]
    P.append(p)
    traj_idx += 1

    if np.linalg.norm(p - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

P = np.hstack(P)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
# Other instance

# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p = p_02
v = np.zeros((2,1))
a = np.zeros((2,1))
# State history
P2 = []
planning_times = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner2.replan((p,v,a))
        planning_times.append(time.time() - start_t)
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    p = trajectory[0][:,traj_idx][:,None]
    v = trajectory[1][:,traj_idx][:,None]
    a = trajectory[2][:,traj_idx][:,None]
    P2.append(p)
    traj_idx += 1

    if np.linalg.norm(p - planner2.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

P2 = np.hstack(P2)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

# Other instance
ax.add_patch(plt.Circle(tuple(p_02), params.R_BOT, color='b'))
ax.scatter(planner2.p_goal[0], planner2.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

# Plot trajectory
ax.scatter(P[0,:], P[1,:], s=10, marker='o', color='b')

# Other instance
ax.scatter(P2[0,:], P2[1,:], s=10, marker='o', color='b')

plt.show()

In [ ]:
# Fixing this so they don't collide.

# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False
trajectory2 = None

# Initialize state
p = p_0
v = np.zeros((2,1))
a = np.zeros((2,1))
p2 = p_02
v2 = np.zeros((2,1))
a2 = np.zeros((2,1))
# State history
P = []
planning_times = []
P2 = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan((p,v,a))
        if trajectory!=None:
            planner2.obstacles = static.copy() + [(trajectory[0][:,0],params.R_BOT),(trajectory[0][:,1],params.R_BOT),(trajectory[0][:,2],params.R_BOT)]
        trajectory2 = planner2.replan((p2,v2,a2))
        if trajectory2!=None:
            planner.obstacles = static.copy() + [(trajectory2[0][:,0],params.R_BOT),(trajectory2[0][:,1],params.R_BOT),(trajectory2[0][:,2],params.R_BOT)]
        planning_times.append(time.time() - start_t)
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    if trajectory!=None:
        p = trajectory[0][:,traj_idx][:,None]
        v = trajectory[1][:,traj_idx][:,None]
        a = trajectory[2][:,traj_idx][:,None]
    P.append(p)
    # Other instance
    if trajectory2!=None:
        p2 = trajectory2[0][:,traj_idx][:,None]
        v2 = trajectory2[1][:,traj_idx][:,None]
        a2 = trajectory2[2][:,traj_idx][:,None]
    P2.append(p2)
    traj_idx += 1

    if np.linalg.norm(p - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

P = np.hstack(P)
P2 = np.hstack(P2)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

# Other instance
ax.add_patch(plt.Circle(tuple(p_02), params.R_BOT, color='b'))
ax.scatter(planner2.p_goal[0], planner2.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in static:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

# Plot trajectory
ax.scatter(P[0,:], P[1,:], s=10, marker='o', color='b')

# Other instance
ax.scatter(P2[0,:], P2[1,:], s=10, marker='o', color='b')

plt.show()

In [ ]:
trajectory

In [ ]:
type(trajectory)

In [ ]:
static.copy() + [(trajectory[0][:,0],params.R_BOT)]

In [ ]:
static